# Question 3

#### Vizualize data in map

In [1]:
import requests
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.9.0
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [2]:
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(url,'lxml')

data = []
columns = []
table = soup.find('table',{'class':'wikitable sortable'})
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
   
    if (index == 0):
        columns = section
    else:
        data.append(section)

ca_df = pd.DataFrame(data = data,columns = columns)
ca_df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)
ca_df = ca_df[ca_df['Borough'] != 'Not assigned'].reset_index(drop=True)
c_df = ca_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
c_df.columns = ['PostalCode', 'Borough', 'Neighbourhood']

In [3]:
geo_data=pd.read_csv("https://cocl.us/Geospatial_data")
geo_data.rename(columns={'Postal Code':'PostalCode'})

for g,c in zip(geo_data['Postal Code'], ca_df['PostalCode']) :
    if(g==c):
        c_df['Latitude']=geo_data['Latitude']
        c_df['Longitude']=geo_data['Longitude']


c_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
add='Toronto'
geoloc=Nominatim(user_agent="foursquare_agent")
loc=geoloc.geocode(add)
latitude=loc.latitude
longitude=loc.longitude
print(latitude,longitude )


43.653963 -79.387207


In [19]:
venue_map=folium.Map(location=[latitude,longitude],zoom_start=12)

folium.Marker(
    [latitude,longitude],
    popup=add,
    icon=folium.Icon(color='red')
    ).add_to(venue_map)

for lat,lng,label in zip(c_df.Latitude, c_df.Longitude, c_df.Neighbourhood):
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
    [lat,lng],
    popup=label,
    icon=folium.Icon(color='blue')
    ).add_to(venue_map)
    
venue_map